In [1]:
import pandas as pd

forex_df = pd.read_csv("forex_dataset.csv")

In [2]:
forex_df.head()

,Year-Month,Export_Quantity_(Kg),Export_Price_(Rs.),High_Grown_Quantity_(Kg),Medium_Grown_Quantity_(Kg),Low_Grown_Quantity_(Kg),Avg_Exchange_Rate,FEE_(USD)
0,2012-01,21824945.0,497.66,5361185.05,3652353.15,14348315.90,113.942069,9.532390e+07
1,2012-02,22448624.0,509.15,5538399.31,3955745.75,13133213.00,114.102042,1.001710e+08
2,2012-03,27702254.0,536.71,6385525.09,4625138.90,17929328.00,200.000000,7.434038e+07
3,2012-04,23290994.0,540.91,7323011.00,5748401.20,16982493.50,200.000000,6.299166e+07
4,2012-05,30004885.0,567.32,8726319.70,5898297.10,18086627.25,200.000000,8.511186e+07


In [3]:
forex_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Year-Month                  144 non-null    object 
 1   Export_Quantity_(Kg)        144 non-null    float64
 2   Export_Price_(Rs.)          144 non-null    float64
 3   High_Grown_Quantity_(Kg)    144 non-null    float64
 4   Medium_Grown_Quantity_(Kg)  144 non-null    float64
 5   Low_Grown_Quantity_(Kg)     144 non-null    float64
 6   Avg_Exchange_Rate           144 non-null    float64
 7   FEE_(USD)                   144 non-null    float64
dtypes: float64(7), object(1)
memory usage: 9.1+ KB


In [4]:
# Create lagged features for production quantities
for lag in [1, 2]:  # t-1 and t-2
    forex_df[f'High_Grown_Quantity_t-{lag}'] = forex_df['High_Grown_Quantity_(Kg)'].shift(lag)
    forex_df[f'Medium_Grown_Quantity_t-{lag}'] = forex_df['Medium_Grown_Quantity_(Kg)'].shift(lag)
    forex_df[f'Low_Grown_Quantity_t-{lag}'] = forex_df['Low_Grown_Quantity_(Kg)'].shift(lag)

In [5]:
# Create lagged features for exported quantities and prices
for lag in [1, 2]:
    forex_df[f'Exported_Quantity_t-{lag}'] = forex_df['Export_Quantity_(Kg)'].shift(lag)
    forex_df[f'Export_Price_t-{lag}'] = forex_df['Export_Price_(Rs.)'].shift(lag)

In [6]:
# Create lagged features for exchange rates
for lag in [1, 2]:
    forex_df[f'Avg_Exchange_Rate_t-{lag}'] = forex_df['Avg_Exchange_Rate'].shift(lag)

In [7]:
# Exported-to-Production Ratio
forex_df['Exported_to_Production_Ratio'] = (
    forex_df['Export_Quantity_(Kg)'] /
    (forex_df['High_Grown_Quantity_(Kg)'] + forex_df['Medium_Grown_Quantity_(Kg)'] + forex_df['Low_Grown_Quantity_(Kg)'])
)

In [8]:
# Export Price Change
forex_df['Export_Price_Change'] = (
    (forex_df['Export_Price_(Rs.)'] - forex_df['Export_Price_(Rs.)'].shift(1)) / forex_df['Export_Price_(Rs.)'].shift(1)
)

In [9]:
forex_df.isnull().sum()

Year-Month                      0
Export_Quantity_(Kg)            0
Export_Price_(Rs.)              0
High_Grown_Quantity_(Kg)        0
Medium_Grown_Quantity_(Kg)      0
Low_Grown_Quantity_(Kg)         0
Avg_Exchange_Rate               0
FEE_(USD)                       0
High_Grown_Quantity_t-1         1
Medium_Grown_Quantity_t-1       1
Low_Grown_Quantity_t-1          1
High_Grown_Quantity_t-2         2
Medium_Grown_Quantity_t-2       2
Low_Grown_Quantity_t-2          2
Exported_Quantity_t-1           1
Export_Price_t-1                1
Exported_Quantity_t-2           2
Export_Price_t-2                2
Avg_Exchange_Rate_t-1           1
Avg_Exchange_Rate_t-2           2
Exported_to_Production_Ratio    0
Export_Price_Change             1
dtype: int64

In [10]:
# Drop rows with NaN
forex_df = forex_df.dropna().reset_index(drop=True)

In [11]:
# Convert 'Year-Month' to DateTime format
forex_df['Year-Month'] = pd.to_datetime(forex_df['Year-Month'], format='%Y-%m')

In [12]:
# Extract Year and Month
forex_df['Year'] = forex_df['Year-Month'].dt.year
forex_df['Month'] = forex_df['Year-Month'].dt.month

In [13]:
# Rearrange columns to place 'Year' and 'Month' as the first and second columns
columns_order = ['Year', 'Month'] + [col for col in forex_df.columns if col not in ['Year', 'Month']]
forex_df = forex_df[columns_order]

# Drop the 'Year-Month' column
forex_df = forex_df.drop(columns=['Year-Month', 'High_Grown_Quantity_t-1', 'Medium_Grown_Quantity_t-1', 'Low_Grown_Quantity_t-1', 
                                  'High_Grown_Quantity_t-2', 'Medium_Grown_Quantity_t-2', 'Low_Grown_Quantity_t-2', 'Exported_Quantity_t-1', 
                                  'Export_Price_t-1', 'Exported_Quantity_t-2', 'Export_Price_t-2', 'Avg_Exchange_Rate_t-1', 
                                  'Avg_Exchange_Rate_t-2', 'Exported_to_Production_Ratio', 'Export_Price_Change'])

In [14]:
forex_df.corr()

,Year,Month,Export_Quantity_(Kg),Export_Price_(Rs.),High_Grown_Quantity_(Kg),Medium_Grown_Quantity_(Kg),Low_Grown_Quantity_(Kg),Avg_Exchange_Rate,FEE_(USD)
Year,1.000000,-0.033605,-0.618792,0.783040,-0.404676,-0.315422,-0.610345,0.737668,-0.157110
Month,-0.033605,1.000000,0.248162,0.052440,-0.159764,-0.109089,0.081172,0.037929,0.263761
Export_Quantity_(Kg),-0.618792,0.248162,1.000000,-0.567988,0.241734,0.238315,0.539830,-0.526925,0.593730
Export_Price_(Rs.),0.783040,0.052440,-0.567988,1.000000,-0.341132,-0.308379,-0.546046,0.949168,-0.102714
High_Grown_Quantity_(Kg),-0.404676,-0.159764,0.241734,-0.341132,1.000000,0.903259,0.665107,-0.317582,0.045261
Medium_Grown_Quantity_(Kg),-0.315422,-0.109089,0.238315,-0.308379,0.903259,1.000000,0.698441,-0.258538,0.004241
Low_Grown_Quantity_(Kg),-0.610345,0.081172,0.539830,-0.546046,0.665107,0.698441,1.000000,-0.548189,0.311676
Avg_Exchange_Rate,0.737668,0.037929,-0.526925,0.949168,-0.317582,-0.258538,-0.548189,1.000000,-0.309872
FEE_(USD),-0.157110,0.263761,0.593730,-0.102714,0.045261,0.004241,0.311676,-0.309872,1.000000


In [15]:
# Save the final dataset
forex_df.to_csv("final_forex_dataset.csv", index=False)